<a href="https://colab.research.google.com/github/sramponi86/CAS_AIS_Text_Summarization/blob/main/CAS_ExtractiveSummarization_WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook demonstrates the process of extractive summarization using AI techniques. Extractive summarization involves selecting important sentences from a text to create a summary. This can be particularly useful in managing large volumes of information efficiently. We will use natural language processing (NLP) libraries and machine learning models to identify and extract key sentences from a given text.

**Important** If you are running this notebook in JupyterHub, please restart the server and choose the environment *ETH_CAS_AIS Jupyter environment for project 5*.

We begin by installing some dependencies. These libraries include tools for processing language data and pre-trained models for generating sentence embeddings, which are numerical representations of sentences' meanings.

In [1]:
!pip install safetensors sentence_transformers ipyplot==1.1.0
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Next, we import the libraries we will be using. `Spacy` is used for sentence tokenization, `SentenceTransformer` for generating sentence embeddings, and `numpy` for mathematical operations.

In [2]:
from spacy.lang.en import English
from sentence_transformers import SentenceTransformer
import numpy as np

We now initialize Spacy with the English language model and add a sentencizer component. This prepares us to split our text into individual sentences.

In [3]:
spacy = English()
spacy.add_pipe("sentencizer")

We now define a function to split texts into sentences. It is crucial for preparing our text for summarization.

In [4]:
def split(text):
  """
  Splits a text into sentences using the Spacy library.

  Parameters
  ----------
  text : str
      The text to be split into sentences.

  Returns
  -------
  list
      The list of sentences in the text.
  """
  processed_text = spacy(text)
  processed_sentences = processed_text.sents

  sentences = []
  for sentence in processed_sentences:
    sentences.append(sentence.text)

  return sentences

We now define a sample text to be summarized. This text will be processed through our summarization pipeline.

In [5]:
text = "You will rejoice to hear that no disaster has accompanied the \
commencement of an enterprise which you have regarded with such evil \
forebodings. I arrived here yesterday, and my first task is to assure \
my dear sister of my welfare and increasing confidence in the success \
of my undertaking. \
I am already far north of London, and as I walk in the streets of \
Petersburgh, I feel a cold northern breeze play upon my cheeks, which \
braces my nerves and fills me with delight. Do you understand this \
feeling? This breeze, which has travelled from the regions towards \
which I am advancing, gives me a foretaste of those icy climes. \
Inspirited by this wind of promise, my daydreams become more fervent \
and vivid. I try in vain to be persuaded that the pole is the seat of \
frost and desolation; it ever presents itself to my imagination as the \
region of beauty and delight. There, Margaret, the sun is for ever \
visible, its broad disk just skirting the horizon and diffusing a \
perpetual splendour. There—for with your leave, my sister, I will put \
some trust in preceding navigators—there snow and frost are banished; \
and, sailing over a calm sea, we may be wafted to a land surpassing in \
wonders and in beauty every region hitherto discovered on the habitable \
globe. Its productions and features may be without example, as the \
phenomena of the heavenly bodies undoubtedly are in those undiscovered \
solitudes. What may not be expected in a country of eternal light? I \
may there discover the wondrous power which attracts the needle and may \
regulate a thousand celestial observations that require only this \
voyage to render their seeming eccentricities consistent for ever. I \
shall satiate my ardent curiosity with the sight of a part of the world \
never before visited, and may tread a land never before imprinted by \
the foot of man. These are my enticements, and they are sufficient to \
conquer all fear of danger or death and to induce me to commence this \
laborious voyage with the joy a child feels when he embarks in a little \
boat, with his holiday mates, on an expedition of discovery up his \
native river. But supposing all these conjectures to be false, you \
cannot contest the inestimable benefit which I shall confer on all \
mankind, to the last generation, by discovering a passage near the pole \
to those countries, to reach which at present so many months are \
requisite; or by ascertaining the secret of the magnet, which, if at \
all possible, can only be effected by an undertaking such as mine. \
These reflections have dispelled the agitation with which I began my \
letter, and I feel my heart glow with an enthusiasm which elevates me \
to heaven, for nothing contributes so much to tranquillise the mind as \
a steady purpose—a point on which the soul may fix its intellectual \
eye. This expedition has been the favourite dream of my early years. I \
have read with ardour the accounts of the various voyages which have \
been made in the prospect of arriving at the North Pacific Ocean \
through the seas which surround the pole. You may remember that a \
history of all the voyages made for purposes of discovery composed the \
whole of our good Uncle Thomas’ library. My education was neglected, \
yet I was passionately fond of reading. These volumes were my study \
day and night, and my familiarity with them increased that regret which \
I had felt, as a child, on learning that my father’s dying injunction \
had forbidden my uncle to allow me to embark in a seafaring life. \
These visions faded when I perused, for the first time, those poets \
whose effusions entranced my soul and lifted it to heaven. I also \
became a poet and for one year lived in a paradise of my own creation; \
I imagined that I also might obtain a niche in the temple where the \
names of Homer and Shakespeare are consecrated. You are well \
acquainted with my failure and how heavily I bore the disappointment. \
But just at that time I inherited the fortune of my cousin, and my \
thoughts were turned into the channel of their earlier bent. \
Six years have passed since I resolved on my present undertaking. I \
can, even now, remember the hour from which I dedicated myself to this \
great enterprise. I commenced by inuring my body to hardship. I \
accompanied the whale-fishers on several expeditions to the North Sea; \
I voluntarily endured cold, famine, thirst, and want of sleep; I often \
worked harder than the common sailors during the day and devoted my \
nights to the study of mathematics, the theory of medicine, and those \
branches of physical science from which a naval adventurer might derive \
the greatest practical advantage. Twice I actually hired myself as an \
under-mate in a Greenland whaler, and acquitted myself to admiration. I \
must own I felt a little proud when my captain offered me the second \
dignity in the vessel and entreated me to remain with the greatest \
earnestness, so valuable did he consider my services."

In [13]:
personal_text = "The Africa Biofortified Sorghum Project is aimed at resolving the problem of hunger and malnutrition. Also, food security contributes to poverty eradication and the support of economic well-being and development. Therefore, the more general aim of the project was to improve the quality of life in Africa and ensure its sustainable development. \
Food security is not always easily achieved in many African countries, where agriculture has to deal with climate specifics including heat and droughts. Despite its rather low nutritional value, sorghum is tolerant to these conditions, which makes it a typical and even “staple” dietary product for many African countries, including Ethiopia and Sudan (Mastandrea, 2009, p. 111). \
As a result, improving this crop was deemed a suitable strategy for enhancing food stability in Africa. Given the fact that the majority of the African population cannot afford dietary supplements, the improvement of a staple food appears to be a wise choice. \
To successfully deliver the program, modern scientific technology and infrastructure for its transfer were required. Also, education for the stakeholders, as well as other means of providing an enabling environment for the change, was necessary. Both of these requirements also needed funding. No single organization could produce all these factors, which is why an alliance was formed to plan and implement the project. \
The project was created by the Africa Harvest Biotech Foundation International that worked in alliance with a number of “private, public and academic organizations” and received funding from the Bill & Melinda Gates Foundation (Mastandrea, 2009, p. 112). The challenge of organizing the collaboration of the organizations was successfully resolved, and a structure for teamwork was developed. Despite this, the partners “found it challenging” to ensure the engagement of stakeholders, their communication, and the information flow between them (Mastandrea, 2009, p. 113). \
In particular, language and cultural barriers were met in Burkina Faso, but means of resolving them (meetings, training) were found. The political aspect of the work was also challenging: before encountering some difficulties with the permits for the experiments, the organizations had not actively sought the support of governmental figures or the society. The lessons were learned, and the project worked towards ensuring information exchange and vital stakeholder engagement and education. Also, the project found that the best way to motivate stakeholders consists in demonstrating them tangible results. \
The challenges related to the technological part of the project have been appropriately met. The transformation system that was developed for the plant became a useful contribution to the bulk of knowledge of genetic engineering, and it can be used for future projects. In general, I think that the project can indeed be replicated in other countries that suffer from a similar issue because similar problems require similar solutions. However, if it is adjusted, it will be more likely to bring positive results in specific settings that can be different. Also, the implementation lessons and the example of the partnership of public and private organizations from various regions can be of use for future projects. \
"

The next cell applies the previously defined `split` function to our text and prints the individual sentences, allowing us to see how the text has been divided.

In [14]:
sentences = split(personal_text)#text)
print("\n\n".join(sentences))

The Africa Biofortified Sorghum Project is aimed at resolving the problem of hunger and malnutrition.

Also, food security contributes to poverty eradication and the support of economic well-being and development.

Therefore, the more general aim of the project was to improve the quality of life in Africa and ensure its sustainable development.

Food security is not always easily achieved in many African countries, where agriculture has to deal with climate specifics including heat and droughts.

Despite its rather low nutritional value, sorghum is tolerant to these conditions, which makes it a typical and even “staple” dietary product for many African countries, including Ethiopia and Sudan (Mastandrea, 2009, p. 111).

As a result, improving this crop was deemed a suitable strategy for enhancing food stability in Africa.

Given the fact that the majority of the African population cannot afford dietary supplements, the improvement of a staple food appears to be a wise choice.

To succe

We now load a pre-trained model to generate embeddings for each sentence. These embeddings capture the semantic meaning of sentences in a high-dimensional space.

In [15]:
model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings = model.encode(sentences)
print(f'This {sentence_embeddings[0].shape[0]}-dimensional vector is an abstract representation of the sentence:')
print(sentences[0])
print(sentence_embeddings[0])

This 384-dimensional vector is an abstract representation of the sentence:
The Africa Biofortified Sorghum Project is aimed at resolving the problem of hunger and malnutrition.
[-1.07875615e-02  1.26994878e-01 -9.67401713e-02  3.11500989e-02
  2.20988970e-02  4.01049256e-02 -8.81249085e-02  4.82486514e-03
 -6.96459487e-02 -2.70730387e-02  2.24157535e-02 -4.95855622e-02
 -5.05971462e-02 -8.44224542e-02 -3.48101258e-02 -1.08739652e-01
  8.84409249e-02  3.86687182e-02 -2.44966056e-02 -8.07002559e-02
  9.46308374e-02  1.60659868e-02  6.30996898e-02  2.10805843e-03
 -2.56524198e-02 -1.99511126e-02 -1.43109066e-02 -5.89062274e-02
 -2.53433175e-02 -1.29279997e-02  1.02433665e-02  8.05621743e-02
  9.62898880e-02  2.34145503e-02 -4.35579084e-02  9.00313333e-02
  4.13799621e-02  5.27604483e-03 -6.08512983e-02 -6.18001670e-02
  5.85442372e-02 -9.51973051e-02  2.11774521e-02 -5.88462427e-02
 -2.16556452e-02 -5.16627952e-02 -9.70807951e-03  3.96011248e-02
 -2.04821583e-02 -5.38165271e-02 -2.4804132

This section clusters the sentence embeddings into groups using KMeans clustering. This helps in identifying sentences with similar meanings, which can be useful for summarization.

In [21]:
from sklearn.cluster import KMeans

k = 10#4

kmeans = KMeans(n_clusters=k, n_init="auto")
cluster_ids = kmeans.fit_predict(sentence_embeddings)

clustered_sentences = {i: [] for i in range(k)}
clustered_embeddings = {i: [] for i in range(k)}
for sentence, embedding, cluster_id in zip(sentences, sentence_embeddings, cluster_ids):
  clustered_sentences[cluster_id].append(sentence)
  clustered_embeddings[cluster_id].append(embedding)

for cluster_id, sentence_list in clustered_sentences.items():
  print(f"Cluster {cluster_id} consists of the following sentences:")
  for sentence in sentence_list:
    print("   " + sentence)

Cluster 0 consists of the following sentences:
   To successfully deliver the program, modern scientific technology and infrastructure for its transfer were required.
   Both of these requirements also needed funding.
   The transformation system that was developed for the plant became a useful contribution to the bulk of knowledge of genetic engineering, and it can be used for future projects.
Cluster 1 consists of the following sentences:
   The Africa Biofortified Sorghum Project is aimed at resolving the problem of hunger and malnutrition.
   Despite its rather low nutritional value, sorghum is tolerant to these conditions, which makes it a typical and even “staple” dietary product for many African countries, including Ethiopia and Sudan (Mastandrea, 2009, p. 111).
Cluster 2 consists of the following sentences:
   Also, education for the stakeholders, as well as other means of providing an enabling environment for the change, was necessary.
   No single organization could produce a

We now compute for each cluster, its *prototypical sentence*. This is the sentence whose embedding is the closest to the cluster's center of mass. In other words, this is the word that is the closest to all other embeddings in the cluster. We begin this by defining two auxiliary functions.

In [22]:
def euclidean_distance(u, v):
  return np.linalg.norm(u - v)

def find_prototypical(centroid, vectors):
  distances = []
  for vector in vectors:
    d = euclidean_distance(centroid, vector)
    distances.append(d)

  index_closest_vector = 0
  closest_dist = distances[0]
  for i, di in enumerate(distances):
    if di < closest_dist:
      index_closest_vector = i
      closest_dist = di

  return index_closest_vector

In [23]:
# Obtain cluster centroids from k-Means
cluter_centroids = kmeans.cluster_centers_

# This dictionary stores the indices of the prototypical sentences
prototypical_sentence_indices = {}

# Loop through clusters
for cluster_id in range(k):

  # Get cluster centroid
  cluster_centroid = cluter_centroids[cluster_id]

  # Get cluster embeddings
  cluster_embeddings = clustered_embeddings[cluster_id]

  # Find the prototypical embedding in this cluster
  prototypical_index = find_prototypical(cluster_centroid, cluster_embeddings)

  # And store its index
  prototypical_sentence_indices[cluster_id] = prototypical_index


# Now we arrange the sentences accordingly
prototypical_sentences = []
for cluster_id in range(k):

  # Get the sentences in this cluster
  cluster_sentences = clustered_sentences[cluster_id]

  # Get the index of the prototypical sentence
  prototypical_sentence_index = prototypical_sentence_indices[cluster_id]

  # Store the prototypical sentence
  prototypical_sentences.append(
    cluster_sentences[prototypical_sentence_index]
  )


# We now have the k summary sentences that we were looking for...
# but they are shuffled as a result of the clustering
original_indices = []
for sent in prototypical_sentences:
  original_indices.append(sentences.index(sent))

# We re-organize them based on their original order in the document
new_indices = np.argsort(original_indices)
prototypical_sentences = [prototypical_sentences[i] for i in new_indices]

extractive_summary_sentences = prototypical_sentences

Finally, we project the sentence embeddings into a 2-dimensional space and color the embeddings according to the clusters they belong.

In [26]:
# Import necessary libraries for PCA and plotting
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Initialize PCA to reduce to 2 dimensions
pca = PCA(n_components=2)

# Apply PCA to the sentence embeddings
sentence_embeddings_2d = pca.fit_transform(sentence_embeddings)

# Prepare colors; if you have more than 8 clusters, extend this list of colors
colors = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'black', 'orange', 'grey', 'violet']

# Plot each sentence embedding in the 2D space, colored by its cluster
plt.figure(figsize=(10, 8))  # Set the figure size
for i, embedding in enumerate(sentence_embeddings_2d):
    plt.scatter(embedding[0], embedding[1], color=colors[cluster_ids[i]], label=f'Cluster {cluster_ids[i]}')

# Optional: add a legend. This might make the plot crowded if there are many points.
# To improve the legend, we're creating custom legend entries to avoid duplicates
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=10, label=f'Cluster {i}') for i, col in enumerate(colors[:k])]
plt.legend(handles=legend_elements, loc='best', title="Clusters")

plt.title('Sentence Embeddings Reduced to 2D by PCA, Colored by Clustering')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()


In [27]:
print('Summary sentences:\n')
for i, sent in enumerate(extractive_summary_sentences, start=1):
  print(i, sent)

Summary sentences:

1 The Africa Biofortified Sorghum Project is aimed at resolving the problem of hunger and malnutrition.
2 As a result, improving this crop was deemed a suitable strategy for enhancing food stability in Africa.
3 To successfully deliver the program, modern scientific technology and infrastructure for its transfer were required.
4 The project was created by the Africa Harvest Biotech Foundation International that worked in alliance with a number of “private, public and academic organizations” and received funding from the Bill & Melinda Gates Foundation (Mastandrea, 2009, p. 112).
5 Despite this, the partners “found it challenging” to ensure the engagement of stakeholders, their communication, and the information flow between them (Mastandrea, 2009, p. 113).
6 In particular, language and cultural barriers were met in Burkina Faso, but means of resolving them (meetings, training) were found.
7 The political aspect of the work was also challenging: before encountering s

## Practical questions

1. Use a different text from your own choice. Identify the sentences that, in your opinion, should appear in the final summary. Then run the summarizer and discuss how well this text summarizer works for this text.

2. Try different numbers of clusters and see the resulting summary. In your opinion, which number of clusters is ideal for your text of choice?